In [ ]:
import os
import zipfile
import requests
import json
from bioservices import BioModels

In [ ]:
s = BioModels()

In [ ]:
def get_all_models(query, page_size=10):
    """
    Retrieve all models matching a query with pagination.

    Args:
        query (str) : The search query to use.
        page_size (int, optional) : Number of models to retrieve per page. Defaults to 10.

    Returns:
        list : A list of all models retrieved.
    """
    models = []
    offset = 0

    while True:
        try:
            search_results = s.search(query, offset=offset)

            if 'models' not in search_results or not search_results['models']:
                print("No more models found. Retrieval complete.")
                break

            models.extend(search_results['models'])
            print(f"Downloaded page {offset // page_size + 1}: retrieved {len(search_results['models'])} models.")
            offset += page_size

        except Exception as e:
            print(f"An error occurred while retrieving models ! See : {e}")
            break

    return models

In [ ]:
def download_model_file(model_id, sbml_url, directory):
    """
    Download the SBML file for a given model.

    Args:
        model_id (str) : model identifier.
        sbml_url (str) : URL pointing to the SBML file.
        directory (str) : local directory used to save file.

    Returns:
        str or None : full path to the downloaded file, or None if an error occurred.
    """
    try :
        sbml_filename = f"{model_id}.xml"
        model_path = os.path.join(directory, sbml_filename)

        response = requests.get(sbml_url)
        if response.status_code == 200:
            with open(model_path, 'wb') as f :
                f.write(response.content)
            print(f"Model {model_id} successfully downloaded.")
        else :
            raise RuntimeError(f"Failed to download model {model_id} ; See error HTTP {response.status_code}")

        return model_path

    except Exception as e:
        print(f"An error occurred while downloading model {model_id} ; See : {e}")
        return None

In [ ]:
def download_model_with_metadata(model_data, base_directory) :
    """
    Download a model and its metadata, then save them into a ZIP file.

    Args :
        model_data (dict) : A dictionary containing model information, including at least 'id' and 'url'.
        base_directory (str) : Path to the root directory where the model should be saved.

    Returns :
        None
    """
    try :
        model_id = model_data['id']
        sbml_url = model_data.get('url', None)
        title = model_data.get('name', "").lower()
        keywords = model_data.get('submitter_keywords', "").lower()
        immun = model_data.get('immun', "").lower()
        
        def contains_keyword(data, keyword) :
            """
            Recursively search for a keyword ('kw') in all string values of a nested element.

            Args :
                data (any) : The data to search (can be dict, list, str).
                keyword (str) : The keyword to search for.

            Returns :
                bool : True if keyword is found. False otherwise.
            """
            if isinstance(data, dict) :
                return any(contains_keyword(v, keyword) for v in data.values())
            elif isinstance(data, list) :
                return any(contains_keyword(item, keyword) for item in data)
            elif isinstance(data, str) :
                return keyword.lower() in data.lower()
            return False

        if not sbml_url :
            print(f"No URL found for model {model_id}.")
            return

        # Retrieve full metadata
        try :
            full_metadata = s.get_model(model_id)
        except Exception as e :
            print(f"Error retrieving full metadata for {model_id} : {e}")
            return

        # Determine destination directory based on content
        if contains_keyword(full_metadata, "immun") :
            directory = os.path.join(base_directory, "immun")
            directory = os.path.join(directory, "Curated_models" if "BIOM" in model_id else "No_Curated_models")
        elif contains_keyword(full_metadata, "T cell") :
            directory = os.path.join(base_directory, "T-cell")
            directory = os.path.join(directory, "Curated_models" if "BIOM" in model_id else "No_Curated_models")
        else :
            return

        os.makedirs(directory, exist_ok=True)

        # Download SBML file
        model_path = download_model_file(model_id, sbml_url, directory)
        if model_path is None :
            return

        # Save metadata as JSON
        metadata_filename = f"{model_id}_metadata.json"
        metadata_path = os.path.join(directory, metadata_filename)
        with open(metadata_path, 'w', encoding='utf-8') as f :
            json.dump(full_metadata, f, ensure_ascii=False, indent=4)

        # Create ZIP file containing model and metadata
        zip_filename = os.path.join(directory, f"{model_id}.zip")
        with zipfile.ZipFile(zip_filename, 'w') as zipf :
            zipf.write(model_path, os.path.basename(model_path))
            zipf.write(metadata_path, os.path.basename(metadata_path))

        # Remove temporary files
        os.remove(model_path)
        os.remove(metadata_path)

        print(f"Model {model_id} and its metadata saved to {zip_filename}")

    except Exception as e :
        print(f"An error occurred while processing model {model_data['id']} : {e}")

In [ ]:
def main() :
    """
    Main function to run whole file.
    """
    query = (
        'boolean AND modelformat:"SBML" AND NOT modellingapproach:"logical model"'
    )

    base_directory = "downloaded_models_boolean"
    os.makedirs(base_directory, exist_ok=True)

    models = get_all_models(query)

    for model_data in models :
        download_model_with_metadata(model_data, base_directory)

In [ ]:
main()